In [26]:
#type:ignore
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications import ResNet50  
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
IMG_SIZE =224

In [4]:
df = pd.read_csv('labels.csv')

In [5]:
image_paths = ['images/'+ name for name in df['name'].values]

In [6]:
y = df['type'].values

In [10]:
y_labels = to_categorical(y,num_classes=5)

In [11]:
y_labels

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [13]:
def load_image(image_path):
  img = cv2.imread(image_path)      # read image
  if img is None:
      print(f"Warning: Unable to load image at {image_path}")
      return None
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)       # convert to RGB
  img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))    # resize for the model
  img = img/255.0      # normalize pixel values to between 0 and 1

  return img

In [15]:
X_images = np.array([load_image(img) for img in image_paths])

In [16]:
X_train, X_temp, y_train, y_temp = train_test_split(X_images, y_labels, test_size=0.2, random_state=42) # training
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # validation and testing

In [27]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip('horizontal'),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1)
]
)

In [28]:
X_train_augmented = data_augmentation(X_train, training = True)

In [17]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))  # loading model without top layers

In [18]:
for layer in base_model.layers:
    layer.trainable = False   # Freezing layers to exclude from training

In [19]:
x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)

In [20]:
output_layer = Dense(5, activation='softmax', name='type')(x)

In [21]:
model = Model(inputs = base_model.input, outputs=output_layer)

In [22]:
model.compile(Adam(learning_rate=0.0001), loss=
        'categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train_augmented, y_train, batch_size=32, epochs=15, validation_data=(X_val, y_val))

Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 766ms/step - accuracy: 0.7647 - loss: 0.8236 - val_accuracy: 0.7746 - val_loss: 0.8451
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 787ms/step - accuracy: 0.7560 - loss: 0.8529 - val_accuracy: 0.7746 - val_loss: 0.8530
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 806ms/step - accuracy: 0.7548 - loss: 0.8422 - val_accuracy: 0.7746 - val_loss: 0.8553
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 823ms/step - accuracy: 0.7595 - loss: 0.8354 - val_accuracy: 0.7746 - val_loss: 0.8527
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 837ms/step - accuracy: 0.7900 - loss: 0.7493 - val_accuracy: 0.7746 - val_loss: 0.8551
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 870ms/step - accuracy: 0.7821 - loss: 0.7978 - val_accuracy: 0.7746 - val_loss: 0.8484
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 882ms/step - accuracy: 0.7860 - loss: 0.7633 - val_accuracy: 0.7746 - val_loss: 0.8555
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 890ms/step - accuracy: 0.7547 - loss: 0.8637 - val_accu

In [30]:
y_pred = model.predict(X_test)
y_test_classes = y_test.argmax(axis=1)
y_pred_classes = y_pred.argmax(axis=1) if y_pred.shape[1] > 1 else y_pred
accuracy_score(y_test_classes, y_pred_classes)

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 451ms/step


0.7361111111111112